In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define the function to integrate

In [ ]:
def func(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return a*x**2 + b*x + c

In [ ]:
def func_integral(x):
    a = 1.01
    b = -3.04
    c = 2.07
    return (a*x**3) / 3 + (b*x**2)/2 + c*x

In [ ]:
def trapezoid_core(f, x, h):
    return 0.5*h*(f(x+h) + f(x))

In [ ]:
def trapezoid_method(f,a,b,N):
    
    #f == function to integrate
    #a == lower lim
    #b == upper lim
    #N == num steps
    
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    Fint = 0.0
    
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        
        return Fint

## Test trapezoid method

a = 0.0
b = 1.0

Answer = func_integral(b) - func_integral(a)
print("True answer = ", Answer)
error_prev = 1.0

for N in range(10,100,10):
    trap_answer = trapezoid_method(func,a,b,N)
    error = (trap_answer - Answer)/Answer
    print("Trapezoid method ", trap_answer, "Error ", error)
    error_prev = error

# Define the core of the simpson's method

In [ ]:
def simpson_core(f,x,h):
    return h*( f(x) + 4*f(x+h) + f(x+2*h))/3

# Define a wrapper function to perform Simpson's Method

In [ ]:
def simpsons_method(f,a,b,N):
    # f == func to integrate
    # a == lower lim of integration
    # b == upper lim
    # N == num of fun evals to use
    # note the number of chunks will be N-1 
    # so if N is odd, then we don't need to adjust 
    # the last segments
    
    # define x values to perfomr simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using simpson's method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
    
    # apply simpson's rule over the last interval
    # if N is even
    if N % 2 == 0:
        Fint += simpson_core(f,x[-2],0.5*h)
        
    return Fint

# Define the Romberg Integration core

In [ ]:
def romberg_core(f,a,b,i):
    
    # we need the difference b-a
    h = b-a
    
    # and the increment between new func evals
    dh = h/2 ** i
    
    # we need the cofactor
    K = h/2 ** (i+1)
    
    # and the func evals
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
    
    # return the answer
    return K*M

# Define a wrapper function to perform Romberg Integration

In [ ]:
def romberg_integration(f,a,b,tol):
    
    # define an interation var
    i = 0
    
    # define a max num of interations
    imax = 1000
    
    # define an error estimate set to a large value
    delta = 100.0*np.fabs(tol)
    
    # set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    # get the zeroth romberg integration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    # interate by 1
    i += 1
    
    while delta > tol:
        
        # find this romberg interation
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        # compute the new fractional error estiamte
        delta = np.fabs( (I[i]-I[i-1])/I[i])
        
        print(i,I[i],I[i-1],delta)
        
        if delta > tol:
            
            i += 1 
            
            if i > imax:
                print("Max interations reached.")
                raise StopIteration("Stopping iterations after ", i)
    
    #return the answer
    return I[i]

# Check the integrals

In [ ]:
Answer = func_integral(1) - func_integral(0)
print(Answer)
print("Trapezoid")
print(trapezoid_method(func,0,1,10))
print("Simpsons Method")
print(simpsons_method(func,0,1,10))
print("Romberg")
tolerance = 1.0e-4
RI = romberg_integration(func,0,1,tolerance)
print(RI, (RI-Answer)/Answer, tolerance)